# Interagindo com Dados Externos

Nesta seção, veremos como usar o Python para interagir com dados externos de diversas fontes. 

## Arquivos em Disco

### Abrindo Arquivos para Leitura

Em anexo, na pasta `./Dados`, nós temos uma cópia do livro [Dom Casmurro][dom_casmurro] de [Machado de Assis][machado_de_assis] obtido no site do [Projeto Gutenberg][gutenberg]. Para abri-lo, usamos o comando `open` do Python.


[dom_casmurro]: https://pt.wikipedia.org/wiki/Dom_Casmurro
[machado_de_assis]: https://pt.wikipedia.org/wiki/Machado_de_Assis
[gutenberg]: http://www.gutenberg.org/wiki/Main_Page

In [2]:
dom_casmurro = open('Dados/machado_de_assis_dom_casmurro.txt')
dom_casmurro

<_io.TextIOWrapper name='Dados/machado_de_assis_dom_casmurro.txt' mode='r' encoding='UTF-8'>

Para ler todo o conteúdo do arquivo, basta usar o método `read`.

In [3]:
dom_casmurro.read()[:1000] # Mostrando só os primeiros 1000 caracteres

"\ufeffThe Project Gutenberg EBook of Dom Casmurro, by Machado de Assis\n\nThis eBook is for the use of anyone anywhere in the United States and most\nother parts of the world at no cost and with almost no restrictions\nwhatsoever.  You may copy it, give it away or re-use it under the terms of\nthe Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org.  If you are not located in the United States, you'll have\nto check the laws of the country where you are located before using this ebook.\n\nTitle: Dom Casmurro\n\nAuthor: Machado de Assis\n\nRelease Date: October 15, 2017 [EBook #55752]\n\nLanguage: Portuguese\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK DOM CASMURRO ***\n\n\n\n\nProduced by Laura Natal Rodriguez & Marc D'Hooghe at Free\nLiterature (online soon in an extended version,also linking\nto free sources for education worldwide ... MOOC's,\neducational materials,...) (Images generously made available\nby the Bibliotheca Nacional Digital Brasil.)\

Ao invés de retornar todo o arquivo como uma `string`, podemos usar o método `readlines` para nos retornar cada linha em uma `string` dentro de uma lista.

In [4]:
dom_casmurro.seek(0) # retorna o início do arquivo
dom_casmurro.readlines()[:10] # só mostra as 10 primeiras linhas

['\ufeffThe Project Gutenberg EBook of Dom Casmurro, by Machado de Assis\n',
 '\n',
 'This eBook is for the use of anyone anywhere in the United States and most\n',
 'other parts of the world at no cost and with almost no restrictions\n',
 'whatsoever.  You may copy it, give it away or re-use it under the terms of\n',
 'the Project Gutenberg License included with this eBook or online at\n',
 "www.gutenberg.org.  If you are not located in the United States, you'll have\n",
 'to check the laws of the country where you are located before using this ebook.\n',
 '\n',
 'Title: Dom Casmurro\n']

Se o arquivo for muito grande, pode ser interessante ler linha por linha e para isso usamos o método `readline`

In [5]:
dom_casmurro.seek(0)

for i in range(10):
    print(dom_casmurro.readline())

﻿The Project Gutenberg EBook of Dom Casmurro, by Machado de Assis



This eBook is for the use of anyone anywhere in the United States and most

other parts of the world at no cost and with almost no restrictions

whatsoever.  You may copy it, give it away or re-use it under the terms of

the Project Gutenberg License included with this eBook or online at

www.gutenberg.org.  If you are not located in the United States, you'll have

to check the laws of the country where you are located before using this ebook.



Title: Dom Casmurro



Ao terminar com o arquivo, devemos sempre fechá-lo para liberar o espaçõ em memória:

In [6]:
dom_casmurro.close()

E podemos verificar se o arquivo foi fechado com sucesso:

In [7]:
dom_casmurro.closed

True

### Escrevendo em Arquivos

O método `open` para abrir arquivos possui outros modos. Acima, utilizamos o modo de leitura `r` que é o padrão. Vamos usar agora o modo `w` de escrita.

In [8]:
meu_texto = open('Dados/meu_texto.txt', 'w')

In [9]:
meu_texto.write('1\n')
meu_texto.writelines(['2\n', '3\n']) # Usado para escrever uma lista de strings

In [10]:
meu_texto.close()

Ao usar o modo `w`, o conteúdo do arquivo é limpo ao ser aberto. Se quiser, podemos abrir de tal forma que só é permitido adicionar novas linhas, mantendo as linhas originais intactas. Para isso, usamos o modo `a` de `append`:

In [11]:
meu_texto = open('Dados/meu_texto.txt', 'a')
meu_texto.writelines(['4\n', '5\n']) 
meu_texto.close()

Vamos abrir o arquivo para leitura e verificar se foi excrito corretamente:

In [12]:
meu_texto = open('Dados/meu_texto.txt')
print(meu_texto.read())
meu_texto.close()

1
2
3
4
5



### Abrindo Arquivos com `with`

Ao abrir arquivos, é possível que esqueçamos de fechar após utiliá-lo. Para evitar isso podemos usar o gerenciador de contexto (*context manager*) `with`, que fecha o arquivo após o término da execução do bloco de código:

In [18]:
with open('Dados/meu_texto.txt') as f:
    print(f.read())
    
print('O arquivo foi fechado?', f.closed)

1
2
3
4
5

O arquivo foi fechado? True


> Quais são as palavras mais exóticas, ou seja, as menos usadas no Dom Casmurro? E as mais usadas? 
>
> Crie um contador de palavras pare responder as perguntas acima. Aproveite também para escrever o resultado em um arquivo no disco. 

## Obtendo Arquivos da Internet 

Em algumas ocasiões, os dados que vamos precisar estará disponibilizado na Internet e precisaremos trazê-los para o disco. Como exemplo, vamos fazer o *download*  dos dados dos passsageiros do Titanic.

In [30]:
from urllib.request import urlretrieve

url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv'

urlretrieve(url, 'Dados/titanic.csv')

('Dados/titanic.csv', <http.client.HTTPMessage at 0x7fcff4605050>)

E agora verificamos se os dados foram trazidos com sucesso:

In [41]:
with open('Dados/titanic.csv') as f:
    for i in range(6):
        print(f.readline()) # as 6 primeiras linhas

survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone

0,3,male,22.0,1,0,7.25,S,Third,man,True,,Southampton,no,False

1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False

1,3,female,26.0,0,0,7.925,S,Third,woman,False,,Southampton,yes,True

1,1,female,35.0,1,0,53.1,S,First,woman,False,C,Southampton,yes,False

0,3,male,35.0,0,0,8.05,S,Third,man,True,,Southampton,no,True



O arquivo foi trazido com sucesso e foi possível abrir e mostra as 6 primeiras linhas. 

Para isso, vamos usar a biblioteca `requests` ([link][requests]), definida como uma "biblioteca HTTP licensiada sob Apache2, escrita em Python, para seres humanos". Como exemplo, vamos obter a cópia do livro Dom Casmurro de Machado de Assis que usamos acima:

[requests]: https://requests.readthedocs.io/pt_BR/latest/index.html

In [19]:
import requests

url = 'http://www.gutenberg.org/cache/epub/55752/pg55752.txt'

r = requests.get(url)
assert r.status_code == 200

No código acima, importamos a biblioteca `requests` e fizemos uma requisição na URL que aponta para o livro Dom Casmurro. Na última linha, usamos um `assert` para verificar se a condição dada é verdadeira. Se fosse falsa, o `assert` retorna um erro do tipo `AssertionError`. Esta é uma ótima forma de verificar se o código está fazendo o que foi planejado para fazer.

A condição `r.status_code == 200` verifica se a resposta da requisição foi retornada com sucesso ([código 200][status_code_200]).

E o conteúdo da resposta, que, neste caso, é o texto do livro, está disponível no atributo `.text`.

[status_code_200]: https://developer.mozilla.org/pt-BR/docs/Web/HTTP/Status/200

In [22]:
r.text[:1000] # Mostrando só os primeiros 1000 caracteres

"\ufeffThe Project Gutenberg EBook of Dom Casmurro, by Machado de Assis\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and most\r\nother parts of the world at no cost and with almost no restrictions\r\nwhatsoever.  You may copy it, give it away or re-use it under the terms of\r\nthe Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org.  If you are not located in the United States, you'll have\r\nto check the laws of the country where you are located before using this ebook.\r\n\r\nTitle: Dom Casmurro\r\n\r\nAuthor: Machado de Assis\r\n\r\nRelease Date: October 15, 2017 [EBook #55752]\r\n\r\nLanguage: Portuguese\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK DOM CASMURRO ***\r\n\r\n\r\n\r\n\r\nProduced by Laura Natal Rodriguez & Marc D'Hooghe at Free\r\nLiterature (online soon in an extended version,also linking\r\nto free sources for education worldwide ... MOOC's,\r\neducational materials,...) (Images generously made avai

## Arquivos Tabulares (CSV, TSV, ...)

## Arquivos JSON

## Arquivos XML 

## Arquivos HTML 

## Bancos de Dados Relacional 

## APIs REST